### Scrapeando Books to Scrap

In [ ]:
# ENTREGABLE - Books to scrape
import requests as req
from bs4 import BeautifulSoup as bs
import string as st
import pandas as pd

print("¡Bienvenidos al Scrapeador de Books to Scrape! by Rubens")
print("Scrapeando https://books.toscrape.com since 2024")
print("Presione cualquier tecla para scrapear ->")
inicio = input("Presione cualquier tecla para scrapear ->")

# Creamos la lista de libros que sera una lista de diccionarios para los elementos del df
lista_libros = []
# Recorremos todas las paginas
for pagina in range(1,51):
    url = f'https://books.toscrape.com/catalogue/page-{pagina}.html'
    html = req.get(url).content
    soup=bs(html, 'html.parser')

    # Sacamos el listado de libros en cada pagina
    libros = soup.find_all('article')
    print(f"\nVoy por la pagina {pagina}\n")
    # Recorremos esos libros uno a uno para scrapear todos sus datos
    
    for libro in libros:
        try:
            # Diccionario donde almacenamos datos de 1 libro
            dicc_libros = {} 
            
            # Url del catalogo del libro de donde sacamos el resto de los datos
            parte_url_libro = libro.find('h3').find('a').attrs['href']
            url_libro = f'https://books.toscrape.com/catalogue/{parte_url_libro}'
            html_interna = req.get(url_libro).content
            libro_soup=bs(html_interna, 'html.parser')
            
            
            # Cojo la cabecera donde tengo 3 de los elementos que me interesan
            cabecera = libro_soup.find('ul',{'class': 'breadcrumb'}).find_all('li')

            # - Titulo del libro
            titulo_libro = cabecera[3].text.strip()
            dicc_libros['titulo_libro'] = titulo_libro

            # - Categoria del libro
            categoria = cabecera[2].text.strip()
            dicc_libros['categoria'] = categoria

            # - Tipo de Producto
            tipo_producto = cabecera[1].text.strip()
            dicc_libros['tipo_producto'] = tipo_producto
            
            
            # - Cojo la Portada del libro
            portada = libro_soup.find('img').attrs['src']
            dicc_libros['portada'] = portada
                    
            # - Precio del libro
            precio = libro_soup.find('p', {'class':'price_color'}).text.strip()
            dicc_libros['precio'] = precio  
                
            # - Cojo la Descripcion del libro, si es que hay.
            descripcion = libro_soup.find_all('div', {'id':'product_description'})
            if len(descripcion) > 0:
                dicc_libros['descripcion'] = libro_soup.find_all('p')[3].text.strip()
            else:
                dicc_libros['descripcion'] = "sin descripcion"
                
                
            # Ahora cojo la tabla donde esta el resto de los elementos
            cuerpo = libro_soup.find('table').find_all('tr')
                    
            # - Universal Product Code
            upc = cuerpo[0].find('td').text.strip()
            dicc_libros['upc'] = upc
            
            # - Precio sin impuestos
            precio_sin_impuestos = cuerpo[2].find('td').text.strip()
            dicc_libros['precio_sin_impuestos'] = precio_sin_impuestos
            
            # - Precio con impuestos
            precio_con_impuestos = cuerpo[3].find('td').text.strip()
            dicc_libros['precio_con_impuestos'] = precio_con_impuestos 
            
            # - Impuestos
            impuestos = cuerpo[4].find('td').text.strip()
            dicc_libros['impuestos'] = impuestos        
                
            # - Sacamos la cadena de Stock   
            stock = cuerpo[5].find('td').text.strip()
            
            if "In stock" in stock:
                # - Tenemos disponibilidad
                dicc_libros['stock'] = 'si'
                # - Cantidad en Stock
                dicc_libros['cantidad'] = int("".join([c for c in stock if c.isdigit()]))
            else:
                # - No tenemos disponibilidad
                dicc_libros['stock'] = 'no'
                dicc_libros['cantidad'] = 0

            # - Cantidad de criticas
            criticas = cuerpo[6].find('td').text.strip()
            dicc_libros['criticas'] = criticas
            
            
            #Appendeamos a nuestra lista de diccionarios de respuesta
            lista_libros.append(dicc_libros) 
        except:
            print(f"{titulo_libro} No se ha podido añadir")
        else:
            print(f"{titulo_libro} Añadido correctamente")

    print(f"\nTodos los libros de la pagina {pagina}, añadidos correctamente\nLlevo {len(lista_libros)} registros guardados")

books_df = pd.DataFrame(lista_libros)  
books_df

### Pruebas y borradores

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import string as st
import pandas as pd
url = f'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
html = req.get(url).content
soup=bs(html, 'html.parser')

In [ ]:
h3_titulos = soup.find_all('ul',{'class': 'breadcrumb'}).find_all('li')[2]

titulos_ofertas = [h3.text.replace('\n','').strip() for h3 in h3_titulos]
titulos_ofertas

In [10]:
h3_titulos = soup.find('ul',{'class': 'breadcrumb'}).find_all('li')[2]
h3_titulos.text.strip()

'Poetry'

In [12]:
stock = "12sdfasdf34"
paco = int("".join([c for c in stock if c.isdigit()]))
paco

1234

In [ ]:
print(soup.prettify())

In [22]:
df[df["descripcion"]=="Sin descripcion"]

,tipo_producto,categoria,titulo_libro,portada,precio,descripcion,upc,precio_sin_impuestos,precio_con_impuestos,impuestos,stock,cantidad,criticas
160,Books,Default,The Bridge to Consciousness: I'm Writing the B...,../../media/cache/39/c7/39c7bebcde5d9643ede71a...,£32.00,Sin descripcion,efc3768127714ec3,£32.00,£32.00,£0.00,si,15,0


In [17]:
print(libro_soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   The Bridge to Consciousness: I'm Writing the Bridge Between Science and Our Old and New Beliefs. | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:30" name="created"/>
  <meta content="
    
" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="../../static/oscar/favicon.